In [1]:
%matplotlib inline
##from skimage import data, io, filters
import os
import numpy as np
from PIL import Image
import matplotlib
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd

JEF notes 8/20/17: Apply this processing for running the chairs140 experiment. All 140 triplet families are included, and only the close condition triplets were added to the database. This can be modified in the "upload stim dictionary to mongo" codeblock.

JEF notes 9/18/17: Adapting this preprocessing for maxdeg1138 dataset. All triplet families are uploaded and added to database. These include "close" and "far" triplets.

JEF notes 3/27/18: Prepping for wave3 of data collection to "patch" missing utterances on existing chairs1k exemplars.

JEF notes 4/12/18: Prepping for wave4 of data collection to "expand" to chairs2k dataset. Goal is to get at least 4 utterances for each of the new triplets

### copy and rename chairs 

In [2]:
alphanum = dict(zip(range(26),string.ascii_lowercase))
conds = ['close','far']
upload_dir = './to_upload_chairs2k'

In [3]:
## path to images
## old_path_to_images = './geometry/chairs_1K_v1/entire_collection_max_deg_sampling' 
path_to_images = './chairs2k'
path_to_close = os.path.join(path_to_images,'closest_nn','data')
path_to_far = os.path.join(path_to_images,'far_enough','data')
all_paths = [path_to_close,path_to_far]

## path -- get list of all triplets
valid = [d for d in os.listdir(path_to_close) if os.path.isdir(os.path.join(path_to_close, d))]
valid = map(int,valid)

# read in the intersection of rxdh and jefan's heuristic filtering of triplets
filtered = 1 # filtered=1 if only the best triplets are used, otherwise set to 0 to use all triplets
if filtered == True:
    _discard = pd.read_csv(os.path.join(path_to_images,'manually_discarded_triplets.txt'))
    discard = list(np.squeeze(_discard.values))
    valid = [i for i in valid if i not in discard]

In [4]:
print 'Total number of discarded triplet families: {}'.format(len(discard))
print 'Total number of valid triplet families: {}'.format(len(valid))

Total number of discarded triplet families: 205
Total number of valid triplet families: 2026


#### crop image to make sure it's square 

In [ ]:
from PIL import Image
runThis = 0
if runThis:
    rw = 224 # final width of images
    rh = 224   
    for cc,cond in enumerate(all_paths):
        for family in os.listdir(cond):
            if (family != 'model_names_of_triplets.txt') & (family in map(str,valid)):
                all_images = os.listdir(os.path.join(cond,family))
                for i, pix in enumerate(all_images):
                    img = Image.open(os.path.join(cond,family,pix))
                    w = np.shape(img)[1] 
                    h = np.shape(img)[0]                     
                    img = img.crop( (int(w/2-rw/2),int(h/2-rh/2),int(w/2+rw/2),int(h/2+rh/2)) )
                    img.thumbnail([224, 224], Image.ANTIALIAS)                    
                    new_filename = conds[cc] + '_' + family + '_' + alphanum[i] + '_' +  all_images[i].split('.')[0].split('_')[1] + '.png'
                    if not os.path.exists(upload_dir):
                        os.makedirs(upload_dir)
                    new_filepath = os.path.join(upload_dir,new_filename)
                    imagefile = open(new_filepath, 'wb')
                    try:
                        img.save(imagefile, "png", quality=90)
                        imagefile.close()
                    except:
                        print "Cannot save user image"                                                        

### upload to s3

In [ ]:
path_to_img = upload_dir
all_files = os.listdir(path_to_img)
all_files = [i for i in all_files if i != '.DS_Store']
num_conds = 2
num_in_triplet = 3
assert len(all_files) == len(valid)*num_conds*num_in_triplet

In [ ]:
import boto
bucket_name = 'shapenet-chairs-triplets-2k' ## options: ['shapenet-chairs-triplets-2k','shapenet_chair_triplets_1k']
runThis = 0
if runThis:
    conn = boto.connect_s3()
    b = conn.get_bucket(bucket_name) ### if bucket already exists, then get_bucket, else create_bucket
    path_to_img = upload_dir
    all_files = os.listdir(path_to_img)
    for a in all_files:
        print a
        if a != '.DS_Store':
            k = b.new_key(a)
            k.set_contents_from_filename(os.path.join(path_to_img,a))
            k.set_acl('public-read')

### build stimulus dictionary

In [ ]:
path_to_img = upload_dir
all_files = os.listdir(path_to_img)
all_files = [i for i in all_files if i != '.DS_Store']
num_conds = 2
num_in_triplet = 3
assert len(all_files) == len(valid)*num_conds*num_in_triplet

In [ ]:
# write out to .js file
text_file = open("/Users/judithfan/language_and_3d/mturk/experiments/stimList_chairs2k.js","w")
for a in all_files:
    tmp = a.split('_')
    line = 'var '+ a.split('.')[0] + ' = {filename: "' + a + '", condition: "' + tmp[0] + '", family: "' + tmp[1] + '", member: "' + tmp[2].split('.')[0] + '", shapenet_id: "' + tmp[3] + '", url: "https://s3.amazonaws.com/shapenet_chair_triplets_1k/' + a + '"};\n'
    text_file.write(line)
text_file.write('var objectList = [\n')    
for i,f in enumerate(all_files):
    if i < len(all_files)-1:
        text_file.write( f.split('.')[0] + ',')   
    else:
        text_file.write( f.split('.')[0])   
text_file.write('];\n')
text_file.write('module.exports = objectList;')
text_file.close()

### prep json to upload stim dictionary to mongo (db = 'stimuli', collection='chairs1k')

In [ ]:
## should each record be a triplet?
## with each field being a list of id's
## target_status_dict = dict(zip(['a','b','c'],['target','distractor1','distractor2']))
def rotate(l, n):
    return l[n:] + l[:n]

In [ ]:
# extract all prefixes (unique triplet families)
print('Generating list of unique triplets...')
_prefix = []
for a in all_files:
    _tmp = a.split('_')[0] + '_' + a.split('_')[1] 
    _prefix.append([i for i in all_files if i.split('_')[0]+'_'+i.split('_')[1]==_tmp])
prefix = [list(x) for x in set(tuple(x) for x in _prefix)]
assert len(prefix)==len(all_files)/3  
assert np.unique([len(p) for p in prefix])[0]==3    

In [ ]:
print('Generating list of triplets and their attributes...')    
# generate pandas dataframe with different attributes
filename = []
condition = []
family = []
member = []
shapenet_id = []
url = []
games = [] # this field keeps track of which games this triplet has been shown in
target_status = [] # whether this object is the target
for v in range(3): # indexes the "version" -- cycles through each object being the target
    target_status_dict = dict(zip(['a','b','c'],rotate(['target','distractor1','distractor2'],v))) # rotate which object is target each time    
    for triplet in prefix:        
        tmp = triplet[0].split('_')
        filename.append(triplet)
        condition.append(triplet[0].split('_')[0])
        family.append(triplet[0].split('_')[1])
        member.append([t.split('_')[2].split('.')[0] for t in triplet])
        shapenet_id.append([t.split('_')[3].split('.')[0] for t in triplet])
        url.append(["https://s3.amazonaws.com/{}/".format(bucket_name) + t for t in triplet])
        games.append([])    
        target_status.append(rotate(['target','distractor1','distractor2'],v))
assert len(target_status)==len(prefix)*3        

In [ ]:
print('Generating pandas dataframe...')    
table = [filename,condition,family,member,shapenet_id,url,games,target_status]
headers = ['filename','condition','family','member','shapenet_id','url','games','target_status']
df = pd.DataFrame(table)
df = df.transpose()
df.columns = headers

In [ ]:
chairs1k_families = map(str,np.arange(1138))
chairs2k_expansion_only = df[~df['family'].isin(chairs1k_families)] ## make sure this logic is right
assert np.intersect1d(np.unique(chairs2k_expansion_only['family'].values),chairs1k_families).size == 0

In [ ]:
## which dataframe to write out ## options: [df,chairs2k_expansion_only]
out_df = chairs2k_expansion_only
## write out dataset to json file
stimdict = out_df.to_dict(orient='records')
import json
dataset_name = 'chairs2k_expansion_only' ## options: ['chairs1k','chairs2k','chairs2k_expansion_only'] 
with open('{}.js'.format(dataset_name), 'w') as fout:
    json.dump(stimdict, fout)

### upload json to mongo (db = 'stimuli', coll = 'chairs2k') April 12 2018¶

JEF notes 4/12/18: Goal is to expand the dataset by getting at least 4 utterances for the new triplets from family 1138-2231. Otherwise exactly the same as previous waves.

In [ ]:
### next todo is to upload this JSON to initialize the new stimulus collection
import json
J = json.loads(open('{}.js'.format(dataset_name),mode='ru').read())

In [ ]:
##assert len(J)==len(all_files)
print 'dataset_name: {}'.format(dataset_name)
print len(J)

In [ ]:
import pandas as pd
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[dataset_name]

In [ ]:
## actually add data now to the database
for (i,j) in enumerate(J):
    if i%500==0:
        print ('%d of %d' % (i,len(J)))
    coll.insert_one(j)

In [ ]:
db.collection_names()

## For patching experiment only: upload json to mongo (db = 'stimuli', coll = 'chairs1k') April 1 2018

JEF notes 3/27/18: Goal is to "patch" the dataset by targeting data collection on triplets for which we are missing utterances so it is as uniform (counterbalanced) as possible. We had been marking the full batch of 70 triplets when retrieving them from the db, which ended up being inaccurate for sessions in which pairs might have dropped out early. So we're going to archive that stimulus collection and rename it "chairs1k_archived." Then we're going to upload a "clone" of chairs1k dataset where the marking is consistent with the groupdata csv (so in other words, only triplets that actually have utterance data are marked).

In [ ]:
### next todo is to upload this JSON to initialize the new stimulus collection
import json
J = json.loads(open('chairs1k_annotated_032718.js',mode='ru').read())

In [ ]:
##assert len(J)==len(all_files)
print len(J)

In [ ]:
import pandas as pd
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db['chairs1k']

In [ ]:
#### if you need to reset the stim db back to the way it was before we needed to do "patching", 
#### only under those circumstances, drop the chairs1k collection


In [ ]:
## actually add data now to the database
for (i,j) in enumerate(J):
    if i%500==0:
        print ('%d of %d' % (i,len(J)))
    coll.insert_one(j)

In [ ]:
coll.find_one()

In [ ]:
a = coll.find({'family':'1'}).sort('condition')
for rec in a:
    print rec['filename']